In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import gc
import lightgbm as lgb
plt.rcParams['figure.figsize'] = [20, 8]
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))

['sample_submission.csv', 'save', 'test.csv', 'train.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
y = train['target']
train.drop(['ID_code', 'target'], axis=1, inplace=True)
test.drop(['ID_code'], axis=1, inplace=True)
gc.collect()

21

****Set different parameters for 5 models. The parameters are chosen almost randomly. Also, we have the same size of the training and test sample, so we will make a prediction for them at the same time. You can add different models, with different parameters and also add more layers before the final prediction.****

In [4]:
param1 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'learning_rate' : 0.1,}
param2 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'num_leaves': 7,
                 'max_depth': 20,
                 'reg_alpha': 0.1,
                 'reg_lambda': 0.1,
                 'learning_rate' : 0.2,}
param3 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'num_leaves': 20,
                 'max_depth': 15,
                 'reg_alpha': 0,
                 'reg_lambda': 0,
                 'learning_rate' : 0.2,}
param4 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'num_leaves': 20,
                 'max_depth': 25,
                 'reg_alpha': 0.5,
                 'reg_lambda': 0.5,
                 'learning_rate' : 0.15,}
param5 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'num_leaves': 30,
                 'max_depth': 100,
                 'reg_alpha': 0.977,
                 'reg_lambda': 0.411,
                 'learning_rate' : 0.083,
                 'min_data_in_leaf':1749,
                 'bagging_fraction ':0.645,
                 'bagging_fraq': 11,
                 'feature_fraction':0.458,
                 'verbose': 200,
                 'min_gain_to_split':0.542,
                 'subsample' : 0.376,
                 'max_bin':638}

In [5]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb

kf = KFold(n_splits=3, random_state = 42,shuffle = True)

oof_clf1 = np.zeros(len(train))
oof_clf2 = np.zeros(len(train))
oof_clf3 = np.zeros(len(train))
oof_clf4 = np.zeros(len(train))
oof_clf5 = np.zeros(len(train))

oof_clf1_t = np.zeros(len(train))
oof_clf2_t = np.zeros(len(train))
oof_clf3_t = np.zeros(len(train))
oof_clf4_t = np.zeros(len(train))
oof_clf5_t = np.zeros(len(train))

for fold_, (trn_idx, val_idx) in enumerate(kf.split(train.values, y.values)):
        print("fold n°{}".format(fold_))
        
        train_data = lgb.Dataset(train.iloc[trn_idx], label = y.iloc[trn_idx])
        valid_data = lgb.Dataset(train.iloc[val_idx], label=y.iloc[val_idx])
        
        model1 = lgb.train(param1, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
        model2 = lgb.train(param2, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
        model3 = lgb.train(param3, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
        model4 = lgb.train(param4, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
        model5 = lgb.train(param5, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)            
        
        oof_clf1[val_idx] = model1.predict(train.iloc[val_idx])
        oof_clf2[val_idx] = model2.predict(train.iloc[val_idx])
        oof_clf3[val_idx] = model3.predict(train.iloc[val_idx])
        oof_clf4[val_idx] = model4.predict(train.iloc[val_idx])
        oof_clf5[val_idx] = model5.predict(train.iloc[val_idx])
        
        oof_clf1_t[val_idx] = model1.predict(test.iloc[val_idx])
        oof_clf2_t[val_idx] = model2.predict(test.iloc[val_idx])
        oof_clf3_t[val_idx] = model3.predict(test.iloc[val_idx])
        oof_clf4_t[val_idx] = model4.predict(test.iloc[val_idx])
        oof_clf5_t[val_idx] = model5.predict(test.iloc[val_idx])
        
        print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf1[val_idx])))
        print("AUC LGB2:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf2[val_idx])))
        print("AUC LGB3:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf3[val_idx])))
        print("AUC LGB4:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf4[val_idx])))
        print("AUC LGB5:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf5[val_idx]))) 

fold n°0
Training until validation scores don't improve for 2000 rounds.
[100]	valid_0's auc: 0.862603
[200]	valid_0's auc: 0.878598
[300]	valid_0's auc: 0.883415
[400]	valid_0's auc: 0.885014
[500]	valid_0's auc: 0.885156
[600]	valid_0's auc: 0.885005
[700]	valid_0's auc: 0.884685
[800]	valid_0's auc: 0.884265
[900]	valid_0's auc: 0.883916
[1000]	valid_0's auc: 0.883828
[1100]	valid_0's auc: 0.883523
[1200]	valid_0's auc: 0.883532
[1300]	valid_0's auc: 0.882967
[1400]	valid_0's auc: 0.882761
[1500]	valid_0's auc: 0.882916
[1600]	valid_0's auc: 0.882729
[1700]	valid_0's auc: 0.882459
[1800]	valid_0's auc: 0.882574
[1900]	valid_0's auc: 0.882586
[2000]	valid_0's auc: 0.882542
[2100]	valid_0's auc: 0.882555
[2200]	valid_0's auc: 0.882642
[2300]	valid_0's auc: 0.882509
[2400]	valid_0's auc: 0.882467
[2500]	valid_0's auc: 0.8826
Early stopping, best iteration is:
[552]	valid_0's auc: 0.885467
Training until validation scores don't improve for 2000 rounds.
[100]	valid_0's auc: 0.864973
[200

[1200]	valid_0's auc: 0.891083
[1300]	valid_0's auc: 0.891083
[1400]	valid_0's auc: 0.891083
[1500]	valid_0's auc: 0.891083
[1600]	valid_0's auc: 0.891083
[1700]	valid_0's auc: 0.891083
[1800]	valid_0's auc: 0.891083
[1900]	valid_0's auc: 0.891083
[2000]	valid_0's auc: 0.891083
[2100]	valid_0's auc: 0.891083
[2200]	valid_0's auc: 0.891083
[2300]	valid_0's auc: 0.891083
[2400]	valid_0's auc: 0.891083
[2500]	valid_0's auc: 0.891083
[2600]	valid_0's auc: 0.891083
Early stopping, best iteration is:
[674]	valid_0's auc: 0.892165
Fold no: 1
AUC LGB1:0.8871385216513094 
AUC LGB2:0.8892682124192708 
AUC LGB3:0.8819479111446212 
AUC LGB4:0.8850780658114323 
AUC LGB5:0.892164860044639 
fold n°2
Training until validation scores don't improve for 2000 rounds.
[100]	valid_0's auc: 0.86585
[200]	valid_0's auc: 0.882259
[300]	valid_0's auc: 0.886924
[400]	valid_0's auc: 0.888044
[500]	valid_0's auc: 0.888052
[600]	valid_0's auc: 0.887699
[700]	valid_0's auc: 0.887287
[800]	valid_0's auc: 0.886934
[90

****Now we will make a data frame with the predictions of the previous models, on which we will make the main prediction****

In [6]:
f_layer_train = pd.DataFrame( {'LGBM1': oof_clf1.ravel(),
     'LGBM2': oof_clf2.ravel(),
     'LGBM3': oof_clf3.ravel(),
      'LGBM4' :oof_clf4.ravel(),
        'LGBM5': oof_clf5.ravel(),
    })

f_layer_test = pd.DataFrame( {'LGBM1': oof_clf1_t.ravel(),
     'LGBM2': oof_clf2_t.ravel(),
     'LGBM3': oof_clf3_t.ravel(),
      'LGBM4' :oof_clf4_t.ravel(),
        'LGBM5': oof_clf5_t.ravel(),
    })

In [10]:
stacked_train_df = pd.concat([train, f_layer_train])

In [11]:
stacked_test_df = pd.concat([test, f_layer_test])

In [7]:
param = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'learning_rate' : 0.1,}

In [8]:
'''
kf = KFold(n_splits=5, random_state = 42,shuffle = True)

oof_clf1_3 = np.zeros(len(train))
oof_clf1_t_3 = np.zeros(len(train))

for fold_, (trn_idx, val_idx) in enumerate(kf.split(f_layer_train.values, y.values)):
        print("fold n°{}".format(fold_))
        train_data = lgb.Dataset(f_layer_train.iloc[trn_idx], label = y.iloc[trn_idx])
        valid_data = lgb.Dataset(f_layer_train.iloc[val_idx], label=y.iloc[val_idx])
        
        model = lgb.train(param, train_data, 10000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
                
        oof_clf1_3[val_idx] = model.predict(f_layer_train.iloc[val_idx])        
        
        oof_clf1_t_3[val_idx] = model.predict(f_layer_test.iloc[val_idx])
        
        print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf1_3[val_idx])))
'''

fold n°0
Training until validation scores don't improve for 2000 rounds.
[100]	valid_0's auc: 0.893353
[200]	valid_0's auc: 0.891959
[300]	valid_0's auc: 0.890988
[400]	valid_0's auc: 0.890297
[500]	valid_0's auc: 0.889389
[600]	valid_0's auc: 0.888577
[700]	valid_0's auc: 0.887853
[800]	valid_0's auc: 0.886979
[900]	valid_0's auc: 0.88625
[1000]	valid_0's auc: 0.885727
[1100]	valid_0's auc: 0.885817
[1200]	valid_0's auc: 0.885176
[1300]	valid_0's auc: 0.88479
[1400]	valid_0's auc: 0.884554
[1500]	valid_0's auc: 0.884146
[1600]	valid_0's auc: 0.883621
[1700]	valid_0's auc: 0.88315
[1800]	valid_0's auc: 0.882772
[1900]	valid_0's auc: 0.882491
[2000]	valid_0's auc: 0.882094
Early stopping, best iteration is:
[31]	valid_0's auc: 0.894378
Fold no: 0
AUC LGB1:0.8943778734868639 
fold n°1
Training until validation scores don't improve for 2000 rounds.
[100]	valid_0's auc: 0.896699
[200]	valid_0's auc: 0.895834
[300]	valid_0's auc: 0.895055
[400]	valid_0's auc: 0.89381
[500]	valid_0's auc: 0.

In [12]:
lgbm_param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': 1
}

In [ ]:
kf = KFold(n_splits=5, random_state = 42,shuffle = True)

oof_clf1_3 = np.zeros(len(train))
oof_clf1_t_3 = np.zeros(len(train))

for fold_, (trn_idx, val_idx) in enumerate(kf.split(stacked_train_df.values, y.values)):
        print("fold n°{}".format(fold_))
        train_data = lgb.Dataset(stacked_train_df.iloc[trn_idx], label = y.iloc[trn_idx])
        valid_data = lgb.Dataset(stacked_test_df.iloc[val_idx], label=y.iloc[val_idx])
        
        model = lgb.train(lgbm_param, train_data, 10000, valid_sets=[valid_data], early_stopping_rounds=2000, verbose_eval=100)
                
        oof_clf1_3[val_idx] = model.predict(stacked_train_df.iloc[val_idx])        
        
        oof_clf1_t_3[val_idx] = model.predict(stacked_test_df.iloc[val_idx])
        
        print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf1_3[val_idx])))

In [9]:
sbm = pd.read_csv('../input/sample_submission.csv', usecols=['ID_code'])
sbm['target'] = oof_clf1_t_3
sbm.head()

,ID_code,target
0,test_0,0.074043
1,test_1,0.256921
2,test_2,0.226807
3,test_3,0.096224
4,test_4,0.035075


In [10]:
sbm.to_csv('submission.csv', index=False)